In [1]:
from data.process import *

2024-07-25 17:28:13.795934: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 17:28:13.803982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 17:28:13.814810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 17:28:13.814840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 17:28:13.822466: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

/home/gakim/gabyproj/InsuranceCrossSelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after first optimization
    'colsample_bytree': 0.25,
    'learning_rate': 0.07,

    # fixed after third optimization
    'max_depth': 64,
    'num_leaves': 4096,
    'min_data_in_leaf': 25,

    # override fixed params
    'n_estimators': 10000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['category']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.888858	valid_0's binary_logloss: 0.24616
[200]	valid_0's auc: 0.890962	valid_0's binary_logloss: 0.24238
[300]	valid_0's auc: 0.891825	valid_0's binary_logloss: 0.241408
[400]	valid_0's auc: 0.891992	valid_0's binary_logloss: 0.241219
[500]	valid_0's auc: 0.892064	valid_0's binary_logloss: 0.241172
[600]	valid_0's auc: 0.892075	valid_0's binary_logloss: 0.241164
[700]	valid_0's auc: 0.891981	valid_0's binary_logloss: 0.241251


In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8921214229378306),
                                     ('binary_logloss', 0.2411262226249931)])})


In [8]:
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]